https://stackoverflow.com/questions/53781634/aggregation-in-pandas

In [1]:
import pandas as pd
import numpy as np

# Question 1
## How to perform aggregation with pandas ?
Expanded aggregation documentation: http://pandas.pydata.org/pandas-docs/stable/groupby.html#aggregation

Aggregating functions are the ones that reduce the dimension of the returned objects. It means output Series/DataFrame have less or same rows like original. Some common aggregating functions are tabulated below:
 

|Function    |Description|
|------------|-----------|
|mean()      |Compute mean of groups|
|sum()       |Compute sum of group values|
|size()      |Compute group sizes|
|count()     |Compute count of group|
|std()       |Standard deviation of groups|
|var()       |Compute variance of groups|
|sem()       |Standard error of the mean of groups|
|describe()  |Generates descriptive statistics|
|first()     |Compute first of group values|
|last()      |Compute last of group values|
|nth()       |Take nth value, or a subset if n is a list|
|min()       |Compute min of group values|
|max()       |Compute max of group values|
```

In [2]:
np.random.seed(123)

choices_a = ['foo', 'baa']
choices_b = ['one', 'two', 'three']
sample_size = 50

df = pd.DataFrame({
    'A' : np.random.choice(choices_a, sample_size),
    'B' : np.random.choice(choices_b, sample_size),
    'C' : np.random.randint(5, size=sample_size),
    'D' : np.random.randint(5, size=sample_size),
    'E' : np.random.randint(5, size=sample_size)
})

display(df.head())

,A,B,C,D,E
0,foo,two,1,0,4
1,baa,two,1,1,0
2,foo,three,2,0,0
3,foo,three,3,2,4
4,foo,three,3,4,1


Aggregation by filtered columns and cython implemented functions: http://pandas.pydata.org/pandas-docs/stable/groupby.html#cython-optimized-aggregation-functions

In [3]:
df1 = df.groupby(['A', 'B'], as_index=False)['C'].sum()

display(df1)

,A,B,C
0,baa,one,7
1,baa,three,15
2,baa,two,19
3,foo,one,17
4,foo,three,27
5,foo,two,21


Aggregate function is using for all columns without specified in `groupby` function, here `A, B` columns:

In [4]:
df2 = df.groupby(['A', 'B'], as_index=False).sum()

display(df2)

,A,B,C,D,E
0,baa,one,7,13,16
1,baa,three,15,20,16
2,baa,two,19,16,14
3,foo,one,17,15,16
4,foo,three,27,19,23
5,foo,two,21,15,18


You can also specify only some columns used for aggregation in a list after `groupby` function:

In [5]:
df3 = df.groupby(['A', 'B'], as_index=False)['C','D'].sum()

display(df3)

<ipython-input-5-113ff2f8afe8>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df.groupby(['A', 'B'], as_index=False)['C','D'].sum()


,A,B,C,D
0,baa,one,7,13
1,baa,three,15,20
2,baa,two,19,16
3,foo,one,17,15
4,foo,three,27,19
5,foo,two,21,15


Same results by using function DataFrameGroupBy.agg: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html

In [6]:
df1 = df.groupby(['A', 'B'], as_index=False)['C'].agg('sum')

display(df1)

,A,B,C
0,baa,one,7
1,baa,three,15
2,baa,two,19
3,foo,one,17
4,foo,three,27
5,foo,two,21


In [7]:
df2 = df.groupby(['A', 'B'], as_index=False).agg('sum')

display(df2)

,A,B,C,D,E
0,baa,one,7,13,16
1,baa,three,15,20,16
2,baa,two,19,16,14
3,foo,one,17,15,16
4,foo,three,27,19,23
5,foo,two,21,15,18


For multiple functions applied for one column use a list of `tuple`s - names of new columns and aggregated functions:

In [8]:
df4 = (df.groupby(['A', 'B'])['C'].agg([('average','mean'),('total','sum')]).reset_index())

display(df4)

,A,B,average,total
0,baa,one,1.400000,7
1,baa,three,1.875000,15
2,baa,two,2.111111,19
3,foo,one,1.888889,17
4,foo,three,2.454545,27
5,foo,two,2.625000,21


If want to pass multiple functions is possible pass `list` of `tuple`s:

In [9]:
df5 = (df.groupby(['A', 'B']).agg([('average','mean'),('total','sum')]))

display(df5)

C               D               E      
            average total   average total   average total
A   B                                                    
baa one    1.400000     7  2.600000    13  3.200000    16
    three  1.875000    15  2.500000    20  2.000000    16
    two    2.111111    19  1.777778    16  1.555556    14
foo one    1.888889    17  1.666667    15  1.777778    16
    three  2.454545    27  1.727273    19  2.090909    23
    two    2.625000    21  1.875000    15  2.250000    18

Then get `MultiIndex` in columns:

In [10]:
display(df5.columns)

MultiIndex([('C', 'average'),
            ('C',   'total'),
            ('D', 'average'),
            ('D',   'total'),
            ('E', 'average'),
            ('E',   'total')],
           )

And for converting to columns, flattening `MultiIndex` use `ma`p with `join`:

In [11]:
df5.columns = df5.columns.map('_'.join)
df5 = df5.reset_index()

display(df5)

,A,B,C_average,C_total,D_average,D_total,E_average,E_total
0,baa,one,1.400000,7,2.600000,13,3.200000,16
1,baa,three,1.875000,15,2.500000,20,2.000000,16
2,baa,two,2.111111,19,1.777778,16,1.555556,14
3,foo,one,1.888889,17,1.666667,15,1.777778,16
4,foo,three,2.454545,27,1.727273,19,2.090909,23
5,foo,two,2.625000,21,1.875000,15,2.250000,18


Another solution is pass list of aggregate functions, then flatten `MultiIndex` and for another columns names use `str.replace`: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.replace.html

In [12]:
df5 = df.groupby(['A', 'B']).agg(['mean','sum'])
    
df5.columns = (df5.columns.map('_'.join)
                  .str.replace('sum','total')
                  .str.replace('mean','average'))
df5 = df5.reset_index()

display(df5)

,A,B,C_average,C_total,D_average,D_total,E_average,E_total
0,baa,one,1.400000,7,2.600000,13,3.200000,16
1,baa,three,1.875000,15,2.500000,20,2.000000,16
2,baa,two,2.111111,19,1.777778,16,1.555556,14
3,foo,one,1.888889,17,1.666667,15,1.777778,16
4,foo,three,2.454545,27,1.727273,19,2.090909,23
5,foo,two,2.625000,21,1.875000,15,2.250000,18


If want specified each column with aggregated function separately pass `dictionary`:

In [13]:
df6 = (df.groupby(['A', 'B'], as_index=False).agg({'C':'sum','D':'mean'}).rename(columns={'C':'C_total', 'D':'D_average'}))

display(df6)

,A,B,C_total,D_average
0,baa,one,7,2.600000
1,baa,three,15,2.500000
2,baa,two,19,1.777778
3,foo,one,17,1.666667
4,foo,three,27,1.727273
5,foo,two,21,1.875000


You can pass custom function too:

In [14]:
def func(x):
    return x.iat[0] + x.iat[-1]

df7 = (df.groupby(['A', 'B'], as_index=False).agg({'C':'sum','D': func}).rename(columns={'C':'C_total', 'D':'D_sum_first_and_last'}))

display(df7)

,A,B,C_total,D_sum_first_and_last
0,baa,one,7,5
1,baa,three,15,6
2,baa,two,19,4
3,foo,one,17,4
4,foo,three,27,0
5,foo,two,21,4


# Question 2

## No DataFrame after aggregation! What happened?
Aggregation by 2 or more columns:

In [15]:
df1 = df.groupby(['A', 'B'])['C'].sum()

display(df1)

A    B    
baa  one       7
     three    15
     two      19
foo  one      17
     three    27
     two      21
Name: C, dtype: int32

First check `Index` and `type` of pandas object:

In [16]:
display(df1.index)

MultiIndex([('baa',   'one'),
            ('baa', 'three'),
            ('baa',   'two'),
            ('foo',   'one'),
            ('foo', 'three'),
            ('foo',   'two')],
           names=['A', 'B'])

In [17]:
display(type(df1))

pandas.core.series.Series

There are 2 solutions how get `MultiIndex Series` to columns:

* add parameter `as_index=False`

In [18]:
df1 = df.groupby(['A', 'B'], as_index=False)['C'].sum()

display(df1)

,A,B,C
0,baa,one,7
1,baa,three,15
2,baa,two,19
3,foo,one,17
4,foo,three,27
5,foo,two,21


use `Series.reset_index`: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.reset_index.html

In [19]:
df1 = df.groupby(['A', 'B'])['C'].sum().reset_index()

display(df1)

,A,B,C
0,baa,one,7
1,baa,three,15
2,baa,two,19
3,foo,one,17
4,foo,three,27
5,foo,two,21


If group by one column:

In [20]:
df2 = df.groupby('A')['C'].sum()

display(df2)

A
baa    41
foo    65
Name: C, dtype: int32

... get `Series` with `Index`:

In [21]:
display(df2.index)

Index(['baa', 'foo'], dtype='object', name='A')

In [22]:
display(type(df2))

pandas.core.series.Series

And solution is same like in `MultiIndex Series`:

In [23]:
df2 = df.groupby('A', as_index=False)['C'].sum()

display(df2)

,A,C
0,baa,41
1,foo,65


In [24]:
df2 = df.groupby('A')['C'].sum().reset_index()

display(df2)

,A,C
0,baa,41
1,foo,65


# Question 3
## How to aggregate mainly strings columns (to lists, tuples, strings with separator)?

In [25]:
np.random.seed(123)
choices_a = ['a', 'b', 'c']
choices_b = ['one', 'two', 'three']
choices_c = ['foo', 'baa']
sample_size = 50

df = pd.DataFrame({
    'A' : np.random.choice(choices_a, sample_size),
    'B' : np.random.choice(choices_b, sample_size),
    'C' : np.random.choice(choices_c, sample_size),
    'D' : np.random.randint(5, size=sample_size)
})

display(df.head())

,A,B,C,D
0,c,three,foo,4
1,b,three,foo,4
2,c,three,baa,4
3,c,three,foo,0
4,a,two,baa,0


Instead of an aggregetion function it is possible to pass `list`, `tuple`, `set` for converting column:

In [26]:
df1 = df.groupby('A')['B'].agg(list).reset_index()

display(df1)

,A,B
0,a,"[two, two, three, three, two, three, one, one,..."
1,b,"[three, two, three, three, three, three, two, ..."
2,c,"[three, three, three, three, one, one, one, on..."


Alternative is use `GroupBy.apply`: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.apply.html

In [27]:
df1 = df.groupby('A')['B'].apply(list).reset_index()

display(df1)

,A,B
0,a,"[two, two, three, three, two, three, one, one,..."
1,b,"[three, two, three, three, three, three, two, ..."
2,c,"[three, three, three, three, one, one, one, on..."


For converting to strings with separator use `.join` only if string column:

In [28]:
df2 = df.groupby('A')['B'].agg(','.join).reset_index()

display(df2)

,A,B
0,a,"two,two,three,three,two,three,one,one,three,tw..."
1,b,"three,two,three,three,three,three,two,two,two,..."
2,c,"three,three,three,three,one,one,one,one,one,th..."


If numeric column use lambda function with `astype` for converting to `string`s: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html

In [29]:
df3 = (df.groupby('A')['D'].agg(lambda x: ','.join(x.astype(str))).reset_index())

display(df3)

,A,D
0,a,"0,1,4,2,3,1,1,0,1,1,2,2,3,2,3,3,2,2"
1,b,"4,0,2,0,0,4,3,4,1,2,1,1,3,0,2"
2,c,"4,4,0,1,0,4,2,2,1,1,3,0,1,3,2,4,3"


Another solution is converting to strings before `groupby`:

In [30]:
df3 = (df.assign(D = df['D'].astype(str)).groupby('A')['D'].agg(','.join).reset_index())

display(df3)

,A,D
0,a,"0,1,4,2,3,1,1,0,1,1,2,2,3,2,3,3,2,2"
1,b,"4,0,2,0,0,4,3,4,1,2,1,1,3,0,2"
2,c,"4,4,0,1,0,4,2,2,1,1,3,0,1,3,2,4,3"


For converting all columns pass no list of column(s) after `groupby`. There is no column `D` because automatic exclusion of 'nuisance' columns, it means all numeric columns are excluded. http://pandas.pydata.org/pandas-docs/stable/groupby.html#automatic-exclusion-of-nuisance-columns

In [31]:
df4 = df.groupby('A').agg(','.join).reset_index()

display(df4)

,A,B,C
0,a,"two,two,three,three,two,three,one,one,three,tw...","baa,baa,foo,foo,foo,baa,baa,baa,baa,baa,foo,ba..."
1,b,"three,two,three,three,three,three,two,two,two,...","foo,foo,foo,baa,baa,baa,foo,baa,foo,foo,baa,fo..."
2,c,"three,three,three,three,one,one,one,one,one,th...","foo,baa,foo,baa,foo,baa,baa,foo,baa,baa,baa,ba..."


So it's necessary to convert all columns into strings, then get all columns:

In [32]:
df5 = (df.groupby('A').agg(lambda x: ','.join(x.astype(str))).reset_index())

display(df5)

,A,B,C,D
0,a,"two,two,three,three,two,three,one,one,three,tw...","baa,baa,foo,foo,foo,baa,baa,baa,baa,baa,foo,ba...","0,1,4,2,3,1,1,0,1,1,2,2,3,2,3,3,2,2"
1,b,"three,two,three,three,three,three,two,two,two,...","foo,foo,foo,baa,baa,baa,foo,baa,foo,foo,baa,fo...","4,0,2,0,0,4,3,4,1,2,1,1,3,0,2"
2,c,"three,three,three,three,one,one,one,one,one,th...","foo,baa,foo,baa,foo,baa,baa,foo,baa,baa,baa,ba...","4,4,0,1,0,4,2,2,1,1,3,0,1,3,2,4,3"


# Question 4
## How to aggregate counts?

In [33]:
choices_a = ['a', 'b', 'c']
choices_b = ['one', 'two', 'three']
choices_c = [np.nan, *choices_b]
choices_d = [np.nan, 1, 2, 3, 4]
sample_size = 50

df = pd.DataFrame({
        'A' : np.random.choice(choices_a, sample_size),
        'B' : np.random.choice(choices_b, sample_size),
        'C' : np.random.choice(choices_c, sample_size),
        'D' : np.random.choice(choices_d, sample_size)
})

display(df.head())

,A,B,C,D
0,b,one,two,NaN
1,b,three,three,2.0
2,c,one,nan,4.0
3,c,two,three,1.0
4,b,two,one,1.0


Function `GroupBy.size` for `size` of each group: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.size.html

In [34]:
df1 = df.groupby('A').size().reset_index(name='COUNT')

display(df1)

,A,COUNT
0,a,15
1,b,22
2,c,13


Function `GroupBy.count` exclude missing values: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.count.html

In [35]:
df2 = df.groupby('A')['C'].count().reset_index(name='COUNT')

display(df2)

,A,COUNT
0,a,15
1,b,22
2,c,13


Function should be used fo multiple columns for count non missing values:

In [36]:
df3 = df.groupby('A').count().add_suffix('_COUNT').reset_index()

display(df3)

,A,B_COUNT,C_COUNT,D_COUNT
0,a,15,15,11
1,b,22,22,14
2,c,13,13,11


Related function `Series.value_counts` return size object containing counts of unique values in descending order so that the first element is the most frequently-occurring element. Excludes `NaN`s values by default. http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html

In [37]:
df4 = (df['A'].value_counts().rename_axis('A').reset_index(name='COUNT'))

display(df4)

,A,COUNT
0,b,22
1,a,15
2,c,13


If you want same output like using function `groupby` + `size` add `Series.sort_index`: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.sort_index.html

In [38]:
df5 = (df['A'].value_counts().sort_index().rename_axis('A').reset_index(name='COUNT'))

display(df5)

,A,COUNT
0,a,15
1,b,22
2,c,13


# Question 5
## How to create new column filled by aggregated values?
Method `GroupBy.transform` returns an object that is indexed the same (same size) as the one being grouped

Pandas documentation for more information: http://pandas.pydata.org/pandas-docs/stable/groupby.html#transformation

In [39]:
np.random.seed(123)

choices_a = ['foo', 'baa']
choices_b = ['one', 'two', 'three']
sample_size = 50

df = pd.DataFrame({
    'A' : np.random.choice(choices_a, sample_size),
    'B' : np.random.choice(choices_b, sample_size),
    'C' : np.random.randint(5, size=sample_size),
    'D' : np.random.randint(5, size=sample_size)
})

display(df.head())


,A,B,C,D
0,foo,two,1,0
1,baa,two,1,1
2,foo,three,2,0
3,foo,three,3,2
4,foo,three,3,4


In [41]:
df['C1'] = df.groupby('A')['C'].transform('sum')
df['C2'] = df.groupby(['A','B'])['C'].transform('sum')


df[['C3','D3']] = df.groupby('A')['C','D'].transform('sum')
df[['C4','D4']] = df.groupby(['A','B'])['C','D'].transform('sum')

display(df.head())

<ipython-input-41-788527e007f8>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df[['C3','D3']] = df.groupby('A')['C','D'].transform('sum')
<ipython-input-41-788527e007f8>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df[['C4','D4']] = df.groupby(['A','B'])['C','D'].transform('sum')


,A,B,C,D,C1,C2,C3,D3,C4,D4
0,foo,two,1,0,65,21,65,49,21,15
1,baa,two,1,1,41,19,41,49,19,16
2,foo,three,2,0,65,27,65,49,27,19
3,foo,three,3,2,65,27,65,49,27,19
4,foo,three,3,4,65,27,65,49,27,19


# Pandas: aggregate when column contains numpy arrays

In [43]:
DF = pd.DataFrame([
    [1,np.array([10,20,30])],
    [1,np.array([40,50,60])], 
    [2,np.array([20,30,40])],
    ], columns=['category','arraydata'])

display(DF)

,category,arraydata
0,1,"[10, 20, 30]"
1,1,"[40, 50, 60]"
2,2,"[20, 30, 40]"


In [45]:
grouped = DF.groupby("category")
aggregate = list((k, v["arraydata"].sum()) for k, v in grouped)
new_df = pd.DataFrame(aggregate, columns=["category", "arraydata"]).set_index("category")

display(new_df)

,arraydata
category,
1,"[50, 70, 90]"
2,"[20, 30, 40]"


In [46]:
result = DF.groupby("category").agg({"arraydata": lambda x: list(x.sum())})
result["arraydata"] = result["arraydata"].apply(np.array)

display(result)

,arraydata
category,
1,"[50, 70, 90]"
2,"[20, 30, 40]"


# Pandas agg function gives different results for numpy std vs nanstd

In [47]:
arr = np.array([
    [1.17136, 1.11816],
    [1.13096, 1.04134],
    [1.13865, 1.03414],
    [1.09053, 0.96330],
    [1.02455, 0.94728],
    [1.18182, 1.04950],
    [1.09620, 1.06686]
])

df = pd.DataFrame(arr, index=['foo']*3 + ['bar']*4, columns=['A', 'B'])

display(df)

,A,B
foo,1.17136,1.11816
foo,1.13096,1.04134
foo,1.13865,1.03414
bar,1.09053,0.96330
bar,1.02455,0.94728
bar,1.18182,1.04950
bar,1.09620,1.06686


In [50]:
g = df.groupby(df.index)

display(g)

In [51]:
g['A'].agg([np.mean, np.median, np.std])

,mean,median,std
bar,1.098275,1.093365,0.064497
foo,1.146990,1.138650,0.021452


In [52]:
g['A'].agg([np.mean, np.median, np.nanstd])

,mean,median,nanstd
bar,1.098275,1.093365,0.064497
foo,1.146990,1.138650,0.021452


In [53]:
g['A'].agg(['mean', 'median', 'std'])

,mean,median,std
bar,1.098275,1.093365,0.064497
foo,1.146990,1.138650,0.021452


In [54]:
g['A'].agg([np.mean, np.median, lambda x: np.std(x)])

,mean,median,<lambda_0>
bar,1.098275,1.093365,0.055856
foo,1.146990,1.138650,0.017516


In [55]:
g['A'].agg([np.mean, np.median, np.std, lambda x: np.std(x, ddof=1)])

,mean,median,std,<lambda_0>
bar,1.098275,1.093365,0.064497,0.064497
foo,1.146990,1.138650,0.021452,0.021452


In [56]:
g['A'].agg([np.mean, np.median, 'std', lambda x: np.std(x, ddof=1)])

,mean,median,std,<lambda_0>
bar,1.098275,1.093365,0.064497,0.064497
foo,1.146990,1.138650,0.021452,0.021452
